# Kili Tutorial: Importing predictions
In this tutorial, we will show how to import predictions (pre-annotations) into Kili in order to help annotators and accelerate the whole annotation process. The goal of this tutorial is to illustrate some basic components and concepts of Kili in a simple way, but also to dive into the actual process of iteratively developing real applications in Kili.

Additionally:

For an overview of Kili, visit https://kili-technology.com You can also check out the Kili documentation https://cloud.kili-technology.com/docs. Our goal is to export labels that can predict whether an image contains a Porsche or a Tesla.

The tutorial is divided into four parts:

1. Understanding the different types of labels
2. Understanding the data model of a label
3. Pushing predictions to Kili
4. Visualizing predictions in Kili

## 1. Understanding the different types of labels
A label is the annotation or combination of all annotations created on an asset. For example, all houses identified on the satellite image, or all the information annotated text on the document.

There are four categories of labels:

- **default**: an ordinary label, made by an annotator
- **prediction**: a pre-annotation, made by a model
- **autosave**: a temporary label, made by the app every minute while annotating
- **review**: a check, carried out by a reviewer

When you export data (see [How to export labels](https://github.com/kili-technology/kili-playground/blob/master/recipes/export_labels.ipynb)), you can find out which category a label belongs to by looking at the field `labelType`. It can take the following values: `PREDICTION`, `DEFAULT`, `AUTOSAVE`,`REVIEW`.

## 2. Understanding the data model of a label
Predictions under push in Kili in the form of python dictionaries. The format of the dictionary to be pushed depends on the type of data (text, image, audio), the machine learning task(s) the project is about (e.g. simple, multiple classification, transcription, named entity recognition, object detection, etc.) and their order. In summary, it depends on the JSON format that describes the interface of your annotation project.

The following cell shows you how to view this JSON. You need to update the credentials `email`, `password` and `api_endpoint` before, or to have set those as global environement variables.

In [ ]:
# Authentication
import os

#!pip install kili # uncomment if you don't have kili installed already
from kili.client import Kili

api_key = os.getenv('KILI_USER_API_KEY')
api_endpoint = os.getenv('KILI_API_ENDPOINT') # If you use Kili SaaS, use the url 'https://cloud.kili-technology.com/api/label/v2/graphql'

kili = Kili(api_key=api_key, api_endpoint=api_endpoint)

Let us create a new project and try to retrieve the label of one asset, to understand the **data model**. With a new empty project, this code will not execute, so juste make sure you use a project with assets already uploaded, or upload some using the `append_many_to_dataset` function as we use in this notebook. To help you, you can also check out the recipe [Create a project](https://github.com/kili-technology/kili-playground/blob/master/recipes/create_project.ipynb).

In [ ]:
title = 'Title Exemple'
input_type='IMAGE'
json_interface = {
    "jobs": {}
}
project = kili.create_project(input_type=input_type , json_interface=json_interface, title=title)

assets = kili.assets(project_id=project['id'])
if assets :
    label=assets[0]['label']

Taking the example of text classification task, here are the fields you will have for the data model for a label :

- `createdAt`: Date of creation of the Label.
- `id`: Unique identifier of the Label in the Kili database.
- `jsonResponse`: List of label annotations. See detail lower.
 - `JOB_ID_1`: First annotation task, as defined in the interface builder.
  - `categories`: Category the Asset (text) belongs to.
   - `name`: Name of the category.
   - `confidence`: Category Confidence Index. The value is between 0 and 100 for a PREDICTION type Label (produced by a model). The value is always 100 for a Label created by a human.
- `labelType`: Type of label used to identify if the label is a prediction (made by a model), an ordinary label (made by an annotator) or a journal (carried out by a reviewer). Can take the following values PREDICTION, DEFAULT, AUTOSAVE or REVIEW.
- `secondsToLabel`: Time spent creating this Label.
- `totalSecondsToLabel`: Time spent creating Labels for this Asset, all Labels combined.

For other types of tasks the fields will be the same except for the `jsonResponse` for which we follow more or less the data model of the Google APIs.

## 3. Pushing predictions to Kili
To make a prediction in Kili, you need 4 pieces of information:

- A project ID
- An external asset ID
- A model name (arbitrary)
- A jsonResponse, in the same format as the labels `jsonResponse` (see above)

We are going to present an example for each category of project, with an interface of project, two assets and the form of the associated `jsonResponse` for one prediction for one of the assets.

We'll also recall the procedure to get to this point, including the creation of a project, the importation of assets into those projects, and, finally, the importation of predictions. This process will be done for every example at the end of the notebook.

In [ ]:
project_examples=[]

### 3.a) Example for single-class classification

In [ ]:
project_examples.append({
'title' :'Spam classification',
'description' : 'Classify text as spam or not',
'input_type' : 'TEXT',
'json_interface' : {
    "jobs": {
        "JOB_0": {
            "mlTask": "CLASSIFICATION",
            "required": 1,
            "isChild": False,
            "content": {
                "categories": {
                    "YES_IT_IS_SPAM": {"name": "Yes, it is spam"},
                    "NO": {"name": "No."}
                },
                "input": "radio"
            }
        }
    }
},

'assets_to_import' : [
    'Lorem ipsum dolor sit amet, consectetur adipiscing elit.', 
    'Vestibulum in elit sit amet turpis sagittis venenatis ac in nisl.'],

'json_response' : {
    "JOB_0": {
        "categories": [{"name": "YES_IT_IS_SPAM", "confidence": 100 }]
    }
},
'model_name' : 'spam-classification-v0.0.1'
}
)

### 3.b) Example for multi-class classification of sentiment in text

In [ ]:
project_examples.append({
'title' : 'Delivery issue classification',
'description' : 'Classify the problems encountered by the users',
'input_type' : 'TEXT',
'json_interface' : {
    "jobs": {
        "JOB_0": {
            "mlTask": "CLASSIFICATION",
            "required": 1,
            "isChild": False,
            "content": {
                "categories": {
                    "DELIVERY": {"name": "Delivery"},
                    "BAD_PRODUCT": {"name": "Bad product"},
                    "OPINION_CHANGED": {"name": "Opinion changed"},
                    "ILLEGAL_PRODUCT": {"name": "Illegal product"}
                },
                "input": "checkbox"
            }
        }
    }
},
'assets_to_import' : [
    'Good morning, I wanted to contact you about my delivery, that is not what was supposed to be', 
    'NOT THE GOOD DELIVERY AND LATE'],

'json_response' : {
    "JOB_0": {
        "categories": [
            { "name": "BAD_PRODUCT", "confidence": 100 },
            { "name": "ILLEGAL_PRODUCT", "confidence": 100 }]
    }
},
'model_name' : 'delivery-problem-classification-v0.0.1'
}
)

### 3.c) Example for object detection in an image

In [ ]:
project_examples.append({
'title' : 'Car brand recognition',
'description' : 'Identify and locate cars',
'input_type' : 'IMAGE',
'json_interface' : {
    "jobs": {
        "JOB_0": {
            "mlTask": "OBJECT_DETECTION",
            "tools": [
                "rectangle"
            ],
            "instruction": "What car brand ?",
            "required": 1,
            "isChild": False,
            "content": {
                "categories": {
                    "TESLA": {"name": "Tesla"},
                    "FERRARI": {"name": "Ferrari"}
                },
                "input": "radio"
            }
        }
    }
},

'assets_to_import' : [
    "https://images.caradisiac.com/logos/3/8/6/7/253867/S0-tesla-enregistre-d-importantes-pertes-au-premier-trimestre-175948.jpg", 
    "https://img.sportauto.fr/news/2018/11/28/1533574/1920%7C1280%7Cc096243e5460db3e5e70c773.jpg"],

'json_response' : {
    "JOB_0": {
        "annotations": [{
            "boundingPoly": [{
                "normalizedVertices": [
                    { "x": 0.16, "y": 0.82},
                    { "x": 0.16, "y": 0.32 },
                    { "x": 0.82, "y": 0.32 },
                    { "x": 0.82, "y": 0.82 }
                ]}
            ],
            "categories": [{ "name": "TESLA", "confidence": 100 }],
        }]
    }
},
'model_name' : 'car-brand-localisation-v0.0.1'
}
)

### 3.d) Example for named entities recognition in a text

In [ ]:
project_examples.append({
'title' : 'NER Task',
'description' : 'Identify Named entities',
'input_type' : 'TEXT',
'json_interface' :  {
    "jobs": {
        "JOB": {
            "mlTask": "NAMED_ENTITIES_RECOGNITION",
            "isChild": False,
            "content": {
                "categories": {
                    "NAME": {"name": "Name"},
                    "NOUN": {"name": "Noun"},
                    "ADJECTIVE": {"name": "Adjective"},
                    "VERB": {"name": "Verb"}
                },
                "input": "radio"
            }
        }
    }
},

'assets_to_import' : [
    "Chicago Bulls General Manager Jerry Krause was standing near a Gatorade cooler with medicine in hand as players and coaches milled around nearby.", 
    "Jordan, handsome and cool in a Nike sweatsuit, peered down at the hefty Krause, who was wearing a blue sweater tucked into high-rise pants."],

'json_response' : {
    "JOB": {
        "annotations": [
            {"categories": [{ "name": "NAME", "confidence": 100 }],
            "beginOffset": 0,
            "content": "Chicago Bulls"},
            {"categories": [{ "name": "NAME", "confidence": 100 }],
            "beginOffset": 30,
            "content": "Jerry Krause"},
            {"categories": [{ "name": "NAME", "confidence": 100 }],
            "beginOffset": 63,
            "content": "Gatorade"},
            {"categories": [{ "name": "NOUN", "confidence": 100 }],
            "beginOffset": 22,
            "content": "Manager"},
            {"categories": [{ "name": "NOUN", "confidence": 100 }],
            "beginOffset": 14,
            "content": "General"},
            {"categories": [{ "name": "NOUN", "confidence": 100 }],
            "beginOffset": 84,
            "content": "medicine"},
            {"categories": [{ "name": "NOUN", "confidence": 100 }],
            "beginOffset": 104,
            "content": "players"},
            {"categories": [{ "name": "NOUN", "confidence": 100 }],
            "beginOffset": 116,
            "content": "coaches"},
            {"categories": [{ "name": "VERB", "confidence": 100 }],
            "beginOffset": 124,
            "content": "milled"},
            {"categories": [{ "name": "VERB", "confidence": 100 }],
            "beginOffset": 43,
            "content": "was standing"},
            {"categories": [{ "name": "NOUN", "confidence": 100 }],
            "beginOffset": 96,
            "content": "hand"},
            {"categories": [{ "name": "NOUN", "confidence": 100 }],
            "beginOffset": 72,
            "content": "cooler"}
        ]
    }
},
'model_name' : 'ner-model-v0.0.1'
}
)

### 3.e) Example for named-entities recognition in a rich text

In [ ]:
project_examples.append({
'title' : 'NER Task',
'description' : 'Identify Named entities',
'input_type' : 'TEXT',
'json_interface' :  {
    "jobs": {
        "JOB": {
            "mlTask": "NAMED_ENTITIES_RECOGNITION",
            "isChild": False,
            "content": {
                "categories": {
                    "NAME": {"name": "Name"},
                    "NOUN": {"name": "Noun"},
                    "ADJECTIVE": {"name": "Adjective"},
                    "VERB": {"name": "Verb"}
                },
                "input": "radio"
            }
        }
    }
},

'assets_to_import' : [[
    {
        'children': [
            {
                'type': 'p',
                'children': [
                    {
                        'id': 'PART1',
                        'bold': True,
                        'underline': True,
                        'text': 'The unanimous Declaration',
                    },
                    {
                        'id': 'PART2',
                        'bold': True,
                        'text': ' of the thirteen United States of America.',
                    },
                    {
                        'id': 'PART3',
                        'text':
                        "When in the Course of human events, it becomes necessary for one people to dissolve the political bands which have connected them with another, and to assume among the powers of the earth, the separate and equal station to which the Laws of Nature and of Nature's God entitle them, a decent respect to the opinions of mankind requires that they should declare the causes which impel them to the separation.",
                    }
                ]
            }
        ]
    }
]],

'json_response' : {
    "JOB": {
        "annotations": [
            {
                "beginId": 'PART1',
                "beginOffset": 14,
                "categories": [{ "name": "NAME", "confidence": 100 }],
                "content": "Declaration of the thirteen United States of America",
                "endId": 'PART2',
                "endOffset": 41,
            },
        ]
    }
},
'model_name' : 'ner-model-v0.0.1'
})

### 3.f) Example for named-entities relations in a text

In [ ]:
project_examples.append({
'title' : 'Named entity relation',
'description' : 'Subject verb complement',
'input_type' : 'TEXT',
'json_interface' : {
    "jobs": {
        "NAMED_ENTITIES_RECOGNITION_JOB": {
            "mlTask": "NAMED_ENTITIES_RECOGNITION",
            "content": {
                "categories": {
                    "SUBJECT": {"name": "Subject"},
                    "VERB": {"name": "Verb"},
                    "COMPLEMENT": {"name": "Complement"}
                },
                "input": "radio"
            },
            "required": 1,
            "isChild": False
        },
        "NAMED_ENTITIES_RELATION_JOB": {
            "mlTask": "NAMED_ENTITIES_RELATION",
            "content": {
                "categories": {
                    "VERB_AND_SUBJECT_S": {
                        "name": "Verb and subject(s)",
                        "startEntities": ["VERB"],
                        "endEntities": ["SUBJECT"]
                    },
                    "VERB_AND_COMPLEMENT_S": {
                        "name": "Verb and complement(s)",
                        "startEntities": ["VERB"],
                        "endEntities": ["COMPLEMENT"]
                    }
                },
                "input": "radio"
            },
            "required": 0,
            "isChild": False
        }
    }
},

'assets_to_import' : [
    "Jordan, handsome and cool in a Nike sweatsuit, peered down at the hefty Krause, who was wearing a blue sweater tucked into high-rise pants.",
    "Chicago Bulls General Manager Jerry Krause was standing near a Gatorade cooler with medicine in hand as players and coaches milled around nearby."],

'json_response' : {
    "NAMED_ENTITIES_RECOGNITION_JOB": {
        "annotations": [
            {"categories": [{ "name": "SUBJECT", "confidence": 100 }],
            "beginOffset": 0,
            "content": "Jordan",
            "mid": "Jordan"},
            {"categories": [{ "name": "VERB", "confidence": 100 }],
            "beginOffset": 84,
            "content": "was wearing",
            "mid": "wearing"},
            {"categories": [{ "name": "VERB", "confidence": 100 }],
            "beginOffset": 111,
            "content": "tucked",
            "mid": "tucked verb"},
            {"categories": [{ "name": "COMPLEMENT", "confidence": 100 }],
            "beginOffset": 96,
            "content": "a blue sweater tucked into high-rise pants",
            "mid": "blue sweater complement"},
            {"categories": [{ "name": "VERB", "confidence": 100 }],
            "beginOffset": 47,
            "content": "peered down",
            "mid": "peered"},
            {"categories": [{ "name": "COMPLEMENT", "confidence": 100 }],
            "beginOffset": 62,
            "content": "the hefty Krause",
            "mid": "Krause complement"},
            {"categories": [{ "name": "SUBJECT", "confidence": 100 }],
            "beginOffset": 62,
            "content": "the hefty Krause",
            "mid": "Krause subject"}
        ]
    },
    "NAMED_ENTITIES_RELATION_JOB": {
        "annotations": [
            {"categories": [{ "name": "VERB_AND_SUBJECT_S", "confidence": 100 }],
            "startEntities": [{ "mid": "peered" }],
            "endEntities": [{ "mid": "Jordan" }]},
            {"categories": [{ "name": "VERB_AND_COMPLEMENT_S", "confidence": 100 }],
            "startEntities": [{ "mid": "peered" }],
            "endEntities": [{ "mid": "Krause complement" }]},
            {"categories": [{ "name": "VERB_AND_SUBJECT_S", "confidence": 100 }],
            "startEntities": [{ "mid": "wearing" }],
            "endEntities": [{ "mid": "Krause subject" }]},
            {"categories": [{ "name": "VERB_AND_COMPLEMENT_S", "confidence": 100 }],
            "startEntities": [{ "mid": "wearing" }],
            "endEntities": [{ "mid": "blue sweater complement" }]}
          ]
    }
},
'model_name' : 'relation-model-v0.0.1'
}
)

### 3.g) Example for object detection with nested classification in an image

In [ ]:
project_examples.append({
'title' : 'Car brand & colors recognition',
'description' : 'Identify brand and colors',
'input_type' : 'IMAGE',
'json_interface' : {
    "jobs": {
        "CLASSIFICATION_JOB": {
            "mlTask": "CLASSIFICATION",
            "content": {
                "categories": {
                    "RED": {"name": "Red"},
                    "BLACK": {"name": "Black"},
                    "WHITE": {"name": "White"},
                    "GREY": {"name": "Grey"}},
                "input": "checkbox"
            },
            "required": 0,
            "isChild": True,
            "instruction": "Color"
        },
        "JOB": {
            "mlTask": "OBJECT_DETECTION",
            "tools": [
                "rectangle"
            ],
            "required": 1,
            "isChild": False,
            "content": {
                "categories": {
                    "TESLA": {
                        "name": "Tesla",
                        "children": ["CLASSIFICATION_JOB"]
                    },
                    "FERRARI": {
                        "name": "Ferrari",
                        "children": ["CLASSIFICATION_JOB"]
                    }
                },
                "input": "radio"
            }
        }
    }
},
'assets_to_import' : [
    "https://img.sportauto.fr/news/2018/11/28/1533574/1920%7C1280%7Cc096243e5460db3e5e70c773.jpg",
    "https://images.caradisiac.com/logos/3/8/6/7/253867/S0-tesla-enregistre-d-importantes-pertes-au-premier-trimestre-175948.jpg"],
'json_response' : {
    "JOB": {
    "annotations": [{
        "boundingPoly": [{
            "normalizedVertices": [
                { "x": 0.09, "y": 0.84 },
                { "x": 0.09, "y": 0.36 },
                { "x": 0.92, "y": 0.36 },
                { "x": 0.92, "y": 0.84 }
            ]}
        ],
        "categories": [{ "name": "FERRARI", "confidence": 100 }],
        "mid": "unique-ferrari",
        "type": "rectangle",
        "children": {
            "CLASSIFICATION_JOB": {
                "categories": [{ "name": "GREY", "confidence": 100 }]
            }
        }
    }]
    }
},
'model_name' : 'car-brand-color-v0.0.1'
}
)

### 3.h) Example for object detection in a video

In [ ]:
project_examples.append({
'title' : 'Car tracking demo',
'description' : 'Identify cars',
'input_type' : 'FRAME',
'json_interface' : {
    "jobs": {
        "JOB": {
            "mlTask": "OBJECT_DETECTION",
            "tools": [
                "rectangle"
            ],
            "required": 1,
            "isChild": False,
            "content": {
                "categories": {
                    "TESLA": {
                        "name": "Tesla",
                        "children": []
                    },
                    "FERRARI": {
                        "name": "Ferrari",
                        "children": []
                    }
                },
                "input": "radio"
            }
        }
    }
},
'assets_to_import' : [
     ['https://storage.googleapis.com/label-public-staging/video1/video1-img000001.jpg',
      'https://storage.googleapis.com/label-public-staging/video1/video1-img000002.jpg',
      'https://storage.googleapis.com/label-public-staging/video1/video1-img000003.jpg',
      'https://storage.googleapis.com/label-public-staging/video1/video1-img000004.jpg',
      'https://storage.googleapis.com/label-public-staging/video1/video1-img000005.jpg']
],
'json_response' : {
  "0": {
    "JOB": {
      "annotations": [
        {
          "boundingPoly": [
            {
              "normalizedVertices": [
                { "x": 0.57, "y": 0.59 },
                { "x": 0.57, "y": 0.73 },
                { "x": 0.68, "y": 0.59 },
                { "x": 0.68, "y": 0.73 }
              ]
            }
          ],
          "categories": [{ "name": "TESLA", "confidence": 100 }],
          "mid": "2021050515590298-31914",
          "score": None,
          "type": "rectangle",
          "isKeyFrame": True
        }
      ]
    }
  },
  "1": {
    "JOB": {
      "annotations": [
        {
          "boundingPoly": [
            {
              "normalizedVertices": [
                { "x": 0.56, "y": 0.58 },
                { "x": 0.56, "y": 0.74 },
                { "x": 0.68, "y": 0.58 },
                { "x": 0.68, "y": 0.74 }
              ]
            }
          ],
          "categories": [{ "name": "TESLA", "confidence": 100 }],
          "mid": "2021050515590298-31914",
          "score": None,
          "type": "rectangle",
          "isKeyFrame": False
        }
      ]
    }
  },
  "2": {
    "JOB": {
      "annotations": [
        {
          "boundingPoly": [
            {
              "normalizedVertices": [
                { "x": 0.55, "y": 0.58 },
                { "x": 0.55, "y": 0.73 },
                { "x": 0.67, "y": 0.58 },
                { "x": 0.67, "y": 0.73 }
              ]
            }
          ],
          "categories": [{ "name": "TESLA", "confidence": 100 }],
          "mid": "2021050515590298-31914",
          "score": None,
          "type": "rectangle",
          "isKeyFrame": False
        }
      ]
    }
  },
  "3": {
    "JOB": {
      "annotations": [
        {
          "boundingPoly": [
            {
              "normalizedVertices": [
                { "x": 0.55, "y": 0.58 },
                { "x": 0.55, "y": 0.72 },
                { "x": 0.67, "y": 0.58 },
                { "x": 0.67, "y": 0.72 }
              ]
            }
          ],
          "categories": [{ "name": "TESLA", "confidence": 100 }],
          "mid": "2021050515590298-31914",
          "score": None,
          "type": "rectangle",
          "isKeyFrame": False
        }
      ]
    }
  },
  "4": {
    "JOB": {
      "annotations": [
        {
          "boundingPoly": [
            {
              "normalizedVertices": [
                { "x": 0.54, "y": 0.58 },
                { "x": 0.54, "y": 0.72 },
                { "x": 0.66, "y": 0.58 },
                { "x": 0.66, "y": 0.72 }
              ]
            }
          ],
          "categories": [{ "name": "TESLA", "confidence": 100 }],
          "mid": "2021050515590298-31914",
          "score": None,
          "type": "rectangle",
          "isKeyFrame": False
        }
      ]
    }
  }
},
'model_name' : 'car-tracker-v0.0.1'
}
)

Finally, we can set up an example for each project live on our account Kili, with the following snipet :

In [ ]:
import time

for project_example in project_examples:
    project = kili.create_project(user_id=kili.auth.user_id,
                                  title=project_example['title'],
                                  description=project_example['description'],
                                  input_type=project_example['input_type'],
                                  json_interface=project_example['json_interface'])
    
    # Wait for the project to be fully created
    time.sleep(10)
    
    assets_to_import = project_example['assets_to_import']
    if len(assets_to_import) == 0:
        continue
    if isinstance(assets_to_import[0], dict) or isinstance(assets_to_import[0], list):
        content_array = ['']*len(assets_to_import)
        json_content_array = assets_to_import
    else:
        content_array = assets_to_import
        json_content_array = None
    kili.append_many_to_dataset(
        project_id=project['id'], 
        content_array=content_array,
        external_id_array=['ex1','ex2'],
        json_content_array=json_content_array)
    
    id_check = kili.create_predictions(
        project_id=project['id'],
        external_id_array=['ex1'],
        model_name_array=[project_example['model_name']],
        json_response_array=[project_example['json_response']])
    
    assert(id_check['id']== project['id']) # just a check to assert that everything is running


## 4. Visualizing predictions in Kili
To verify that the prediction in question was indeed pushed in Kili, you can go to https://cloud.kili-technology.com/label/projects/[PROJECT_ID]/dataset/labels?currentPage=1&pageSize=50. You should get a new `PREDICTION` line like this:

![update settings](img/import_predictions-visualize_prediction.png)

## Summary
In this tutorial, we accomplished the following:

We introduced the concept of Kili interface settings and the 4 different kind of labels (`DEFAULT`, `PREDICTION`, `AUTOSAVE`,`REVIEW`). We demonstrated how to retrieve a `DEFAULT` label and how to create a `PREDICTION` label. If you enjoyed this tutorial, check out the other Recipes for other tutorials that you may find interesting, including demonstrations of how to use Kili.

You can also visit the Kili website or Kili documentation for more info!

